In [2]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

df_stockton_ca_1h_2025 = pd.read_sql("select * from auto.test_summary_reporting where collection_set_id = 12621;",
                                     con=os.getenv('RSR_SVC_CONN'))

df_stockton_ca_1h_2025.head()

,product_period_id,product_period,collection_set_id,collection_set,test_event_id,device_event_id,collection_area_id,collection_type_id,carrier_id,carrier,...,access_success,task_success,avg_lte_rsrp,avg_lte_rssnr,avg_nr_ss_rsrp,avg_nr_ss_sinr,ts_prod_insert_time,ts_prod_update_time,ts_reporting_insert_time,collection_set_status_id_tsr
0,37,2025-1H,12621,Stockton-CA_2025-1H,202414351,6758976490,20,1,4,Verizon,...,1.0,1.0,-96.000000,12.0,NaN,NaN,2025-02-05 22:31:29.036038,2025-02-05 22:31:29.036038,2025-02-27 04:30:27.451048+00:00,20
1,37,2025-1H,12621,Stockton-CA_2025-1H,202414352,6758976555,20,1,478,Dish,...,1.0,1.0,NaN,NaN,-96.4,8.0,2025-02-05 22:31:27.645523,2025-02-05 22:31:27.645523,2025-02-27 04:30:27.451048+00:00,20
2,37,2025-1H,12621,Stockton-CA_2025-1H,202414353,6758976568,20,1,478,Dish,...,1.0,1.0,NaN,NaN,-97.0,8.0,2025-02-05 22:31:27.647218,2025-02-05 22:31:27.647218,2025-02-27 04:30:27.451048+00:00,20
3,37,2025-1H,12621,Stockton-CA_2025-1H,202414355,6758976596,20,1,1,AT&T,...,NaN,NaN,-68.666667,14.0,NaN,NaN,2025-02-05 22:31:27.401706,2025-02-05 22:31:27.401706,2025-02-27 04:30:27.451048+00:00,20
4,37,2025-1H,12621,Stockton-CA_2025-1H,202414356,6758976651,20,1,1,AT&T,...,1.0,1.0,-87.600000,7.0,NaN,NaN,2025-02-05 22:31:27.400902,2025-02-05 22:31:27.400902,2025-02-27 04:30:27.451048+00:00,20
